In [20]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas para manipulación y análisis de datos en Python.
# Importar librerías para web scraping y manipulación de datos

from bs4 import BeautifulSoup
import requests
# Importar librerías para automatización de navegadores web con Selenium

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
# ChromeDriverManager gestiona la instalación del controlador de Chrome

from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

import random
from datetime import datetime
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings("ignore")

import os
import json


In [21]:
# CONFIGURACION

execution_number = 4
initial_counter = 0
incremental = 500

In [22]:
def get_url(driver, path, accept_cookies = False): 

    url = f"https://www.imdb.com/{path}"

    print(f"getting url with {path}...")
    driver.get(url)
    print("url loaded")
    # driver.maximize_window()
    if accept_cookies:
        sleep(1)
        # Aceptar las cookies PAG IMDB
        try:
            driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
            print("cookies accepted")
        except:
            print("cookies haven't been accepted")

    return driver
    


In [23]:
def getting_name_score(driver, id):
    
    error = None

    #nombre:
    try:
        name = driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text
    except:
        name = f"Error nombre para la pelicula con id: {id}"
        error = id
                
    #puntuacion:
        
    try: 
        score = driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text
                
    except:  
        score = "--"

    return name, score, error

In [24]:
def getting_directors_scripters(driver):
        directors = "DESCONOCIDO"
        scripters = "DESCONOCIDO"

        for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
       

            
            
            try:
                nombre = elemento.find_element("css selector", "span").text

                if nombre == "Director":
                

                    # Seleccionar director
                    try:          
                        directors = [e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""]

                    except:
                        directors = "DESCONOCIDO"
             
                    

                elif nombre == "Writers":
              
                    
                    # Seleccionar Guionista
                    try:          
                        scripters = [e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""]
                   
                    except:
                        scripters = "DESCONOCIDO"
                     
            except:
                continue


        return directors, scripters

In [25]:
def getting_plot(driver):

    try:      
        return driver.find_element("css selector", 'span[data-testid="plot-xl"]').text
                    
    except:
            
        return "DESCONOCIDO"
    

In [26]:
def getting_time(driver):
            
    # Seleccionar DURACION        
    try:   
        
        busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
            
        if "min" in busqueda_duracion or "h" in busqueda_duracion:
            return busqueda_duracion
                
        else:
            return "--"
        
    except:
        
        return "--"

In [27]:
def getting_cast(driver, dict_actores):
    try:
        for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
            actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
            
            for element in actor[0:10]:
            
                nombre = element.text
                    
                if nombre not in dict_actores.keys():
                    
                    dict_actores[nombre]  = element.get_attribute("href")

                    driver.find_element("css selector",'section[data-testid="title-cast"]')
            
    except:
        dict_actores[nombre]  = "desconocido"
    return dict_actores

In [28]:
INCREMENTAL_PATH = "incremental"
TOTAL_PATH = "total"

options = webdriver.FirefoxOptions()
options.page_load_strategy = "eager"
driver = webdriver.Firefox(options)

driver = get_url(driver, "", True)

films_dict = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}
actors_dict = {}
# diccionario_actores = {"Actor":[], "Fehca_nacimiento":[], "Conocido_por:":[], "Que_hace":[], "Premios":[]}
df_films = pd.DataFrame(films_dict)

errors = 0

times_dict = { "id": [], "time_sec": []}

counter = initial_counter
last_checkpoint = None
ids_with_error = []

union_peliculas = pd.read_csv("../INFO_PELIS/cortos_2021.csv")
lista_ID = union_peliculas["ID"].to_list()

for id in lista_ID[counter:counter+incremental]:
    start_time = datetime.now()
    error = True
    while error: 
        try:
            driver = get_url(driver, f"title/{id}")

            error = False
        except:
            print(f"Error getting {id}")
            if errors > 10:
                print(f"Fatal error")
                break
            errors += 1
            sleep(random.randint(2,20))

    errors = 0
    # sleep(1)
    print(f"Getting attributes for {id}")
    name, score, error_id = getting_name_score(driver, id)
    if error_id:
        print(f"error obtaining name for id {id}")
        ids_with_error.append(error_id)
    films_dict["Nombre de la película"].append(name)
    films_dict["Puntuacion_IMDB"].append(score)
    directors, scripters = getting_directors_scripters(driver)
    films_dict["Dirección"].append(directors)
    films_dict["Guionistas"].append(scripters)
    films_dict["Argumento"].append(getting_plot(driver))
    films_dict["Duración"].append(getting_time(driver))
    print(f"Getting cast for {id}")
    actors_dict = getting_cast(driver, actors_dict)

    if counter % 100 == 0:
        print("Saving checkpoint...")
        df_films = pd.DataFrame(films_dict)
        df_films.to_csv(f"{INCREMENTAL_PATH}/patri_films_{initial_counter}_{counter}.csv")
        df_actors = pd.DataFrame({"name": actors_dict.keys(), "link": actors_dict.items()})
        df_actors.to_csv(f"{INCREMENTAL_PATH}/patri_actors_{initial_counter}_{counter}.csv")
        if last_checkpoint:
            os.remove(f"{INCREMENTAL_PATH}/patri_films_{initial_counter}_{last_checkpoint}.csv")
            os.remove((f"{INCREMENTAL_PATH}/patri_actors_{initial_counter}_{last_checkpoint}.csv"))
        last_checkpoint = counter
        driver.close()
        driver = webdriver.Firefox(options)
        driver = get_url(driver, "", True)
    
    counter += 1
    end_time = datetime.now()
    times_dict["id"].append(id)
    times_dict["time_sec"].append((end_time-start_time).seconds)
    print(f"time to obtain {id} with position {counter-1} was {(end_time-start_time).seconds} seconds")
    # break

df_times = pd.DataFrame(times_dict)
df_times.to_csv(f"{TOTAL_PATH}/{execution_number}_times_extraction.csv")
df_films = pd.DataFrame(films_dict)
df_films.to_csv(f"{TOTAL_PATH}/{execution_number}_patri_films_total.csv")
df_actors = pd.DataFrame({"name": actors_dict.keys(), "link": actors_dict.items()})
df_actors.to_csv(f"{TOTAL_PATH}/{execution_number}_patri_actors_total.csv")
with open(f"{TOTAL_PATH}/{execution_number}_ids_with_errors.txt", "w") as file:
    json.dump(ids_with_error, file)
driver.close()

getting url with ...
url loaded
cookies accepted
getting url with title/tt10011052...
url loaded
Getting attributes for tt10011052
Getting cast for tt10011052
Saving checkpoint...
getting url with ...
url loaded
cookies accepted
time to obtain tt10011052 with position 0 was 9 seconds
getting url with title/tt10033714...
url loaded
Getting attributes for tt10033714
Getting cast for tt10033714
time to obtain tt10033714 with position 1 was 2 seconds
getting url with title/tt10040326...
url loaded
Getting attributes for tt10040326
Getting cast for tt10040326
time to obtain tt10040326 with position 2 was 3 seconds
getting url with title/tt10048746...
url loaded
Getting attributes for tt10048746
Getting cast for tt10048746
time to obtain tt10048746 with position 3 was 2 seconds
getting url with title/tt10049134...
url loaded
Getting attributes for tt10049134
Getting cast for tt10049134
time to obtain tt10049134 with position 4 was 3 seconds
getting url with title/tt10049710...
url loaded
Get

In [29]:
df_times[["time_sec"]].mean()

time_sec    3.974
dtype: float64